### Download dataset
<b>Dataset location: </b>http://files.grouplens.org/datasets/movielens/ml-latest-small.zip

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName('Use Collaborative Filtering for movie recommendations') \
    .getOrCreate()

rawData = spark.read\
            .format('csv')\
            .option('header', 'true')\
            .load('data/movielens/ratings.csv')

In [ ]:
rawData.toPandas().head()

#### Pick all columns except the timestamp

In [ ]:
from pyspark.sql.functions import col

dataset = rawData.select(col('userId').cast('int'), 
                         col('movieId').cast('int'), 
                         col('rating').cast('float')
                        )

dataset.toPandas().head()

#### Check the distribution of rating in the dataset

In [ ]:
dataset.select('rating').toPandas().describe()

#### Split into training and test data sets

In [ ]:
(trainingData, testData) = dataset.randomSplit([0.8, 0.2])

### Define the Collaborative Filtering model
Uses the Alternating Least Squares algorithm to learn the latent factors
* <b>maxIter: </b>The maximum number of iterations to run
* <b>regParam: </b>Specifies the regularization parameter in ALS (defaults to 1.0)
* <b>coldStartStrategy: </b> Strategy for handling unknown or new users/items during prediction (which was not encountered in training). Options are 'drop' and 'nan'. We will drop unknown users/items from the predictions

In [ ]:
from pyspark.ml.recommendation import ALS

als = ALS(maxIter=5, 
          regParam=0.1, 
          userCol='userId', 
          itemCol='movieId', 
          ratingCol='rating',
          coldStartStrategy='drop')

#### Build the ALSModel using the model definition and training data

In [ ]:
model = als.fit(trainingData)

#### Get the predictions for the test data

In [ ]:
predictions = model.transform(testData)
predictions.toPandas().head()

#### Compare the distribution of values for ratings and predictions

In [ ]:
predictions.select('rating', 'prediction').toPandas().describe()

#### Get the Root Mean Square Error on the test data

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(metricName='rmse', 
                                labelCol='rating',
                                predictionCol='prediction')

rmse = evaluator.evaluate(predictions)
print('RMSE = ', rmse)

#### The ALS model can be used to get predictions for all users
Specify the number of predictions you would like for each user

In [ ]:
userRecsAll = model.recommendForAllUsers(3)
userRecsAll.show()

#### View the recommendations
For each userId there is a list of tuples representing a movieId and it's rating for the user

In [ ]:
userRecsAll.toPandas().head()

#### Get the top user recommendations for each movie
* The users who are most likely to like a particular movie
* Get the top 3 users

In [ ]:
movieRecsAll = model.recommendForAllItems(3)
movieRecsAll.toPandas().head()

#### Get recommendations for a subset of users
* Start off by creating a list of users who make up our subset
* Convert that list to a dataframe which will be used shortly

In [ ]:
from pyspark.sql.types import IntegerType

usersList = [148, 463, 267]
usersDF = spark.createDataFrame(usersList, IntegerType()).toDF('userId')

usersDF.show()

#### Use the recommendForUserSubset function
This gets the recommendations for specific users

In [ ]:
userRecs = model.recommendForUserSubset(usersDF, 5)
userRecs.toPandas()

#### Extract recommendations for specific user
* We get a list comprising a Row object which in turn contains a list of Rows
* To get the movie names from the movieIds so we will need to perform some transformations

In [ ]:
userMoviesList = userRecs.filter(userRecs.userId == 148)\
.select('recommendations')

userMoviesList.collect()

#### Extract the list of recommendations
We get the list of Rows contining the movieId and rating for the user

In [ ]:
moviesList = userMoviesList.collect()[0].recommendations
moviesList

#### Create a DataFrame containing the movieId and rating as columns
Use the moviesList created previously

In [ ]:
moviesDF = spark.createDataFrame(moviesList)
moviesDF.toPandas()

#### The movie names are stored in a csv file called movies.csv
Load that into another dataframe

In [ ]:
movieData = spark.read.csv('data/movielens/movies.csv',
                              header=True,
                              ignoreLeadingWhiteSpace= True)
movieData.toPandas().head()

In [ ]:
recommendedMovies = movieData.join(moviesDF, on=['movieId'])\
.orderBy('rating', ascending=False)\
.select('title', 'genres', 'rating')

recommendedMovies.toPandas()

In [ ]:
from pyspark.sql.types import IntegerType

def getRecommendationsForUser(userId, numRecs):
    
    usersDF = spark.\
    createDataFrame([userId], IntegerType()).\
    toDF('userId')
    
    userRecs = model.recommendForUserSubset(usersDF, numRecs)
    
    moviesList = userRecs.collect()[0].recommendations
    moviesDF = spark.createDataFrame(moviesList)
    
    recommendedMovies = movieData.join(moviesDF, on=['movieId'])\
    .orderBy('rating', ascending=False)\
    .select('title', 'genres', 'rating')
    
    return recommendedMovies

In [ ]:
recommendationsForUser = getRecommendationsForUser(219, 10)
recommendationsForUser.toPandas()